<a href="https://colab.research.google.com/github/16Jayavelrajan/Chatbot/blob/main/HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!apt-get update -y
!apt-get install -y openmpi-bin openmpi-common libopenmpi-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,003 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

In [6]:
%%writefile mpi_hello.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv); // Initialize MPI

    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank); // Get process rank

    int world_size;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size); // Get total number of processes

    printf("Hello from processor %d out of %d processors!\n", world_rank, world_size);

    MPI_Finalize(); // Finalize MPI
    return 0;
}


Writing mpi_hello.c


In [7]:
!mpicc -o mpi_hello mpi_hello.c


In [10]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_hello


Hello from processor 3 out of 4 processors!
Hello from processor 0 out of 4 processors!
Hello from processor 1 out of 4 processors!
Hello from processor 2 out of 4 processors!


In [11]:
%%writefile mpi_send_recv.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    if (rank == 0) {
        int data = 42;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent %d to Process 1\n", data);
    }
    else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received %d from Process 0\n", received_data);
    }

    MPI_Finalize();
    return 0;
}


Overwriting mpi_send_recv.c


In [12]:
!mpicc -o mpi_send_recv mpi_send_recv.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_send_recv


Process 0 sent 42 to Process 1
Process 1 received 42 from Process 0


In [13]:
%%writefile mpi_bsend.c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int buffer_size = MPI_BSEND_OVERHEAD + sizeof(int);
    void* buffer = malloc(buffer_size);
    MPI_Buffer_attach(buffer, buffer_size);

    if (rank == 0) {
        int data = 42;
        printf("Process 0: Sending %d to Process 1 using MPI_Bsend\n", data);
        MPI_Bsend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
    }
    else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1: Received %d from Process 0\n", received_data);
    }

    MPI_Buffer_detach(&buffer, &buffer_size);
    free(buffer);

    MPI_Finalize();
    return 0;
}


Writing mpi_bsend.c


In [14]:
!mpicc -o mpi_bsend mpi_bsend.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_bsend


Process 0: Sending 42 to Process 1 using MPI_Bsend
Process 1: Received 42 from Process 0


In [15]:
%%writefile mpi_nonblocking.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    MPI_Request request;
    MPI_Status status;

    if (rank == 0) {
        int data = 42;
        printf("Process 0: Sending %d to Process 1 (Non-blocking)\n", data);
        MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
        MPI_Wait(&request, &status);  // Ensuring completion before exit
    }
    else if (rank == 1) {
        int received_data;
        MPI_Irecv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, &request);
        MPI_Wait(&request, &status);  // Ensure the message is received before printing
        printf("Process 1: Received %d from Process 0 (Non-blocking)\n", received_data);
    }

    MPI_Finalize();
    return 0;
}


Writing mpi_nonblocking.c


In [16]:
!mpicc -o mpi_nonblocking mpi_nonblocking.c
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_nonblocking


Process 0: Sending 42 to Process 1 (Non-blocking)
Process 1: Received 42 from Process 0 (Non-blocking)
